In [1]:
#! pip install -U langchain-google-genai langchain langchain-community langchain-experimental langchain-openai langchain-pinecone langchain-text-splitters -q --user

In [1]:
# %pip install lark --user --quiet

# Langchain
https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html
https://python.langchain.com/v0.2/docs/how_to/chatbots_retrieval/#document-chains
### create_retrieval_chain
https://python.langchain.com/v0.1/docs/use_cases/chatbots/retrieval/
### Chroma
https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
### Pdf loader
https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
### Google Genai
https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/
### SelfQueryRetriever
https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/self_query/
### multiple_chains
https://python.langchain.com/v0.1/docs/expression_language/cookbook/multiple_chains/

# Vertexai
https://cloud.google.com/vertex-ai/docs/general/custom-service-account

# GeminiAI
https://ai.google.dev/gemini-api/docs/api-key

# Chroma
https://docs.trychroma.com/integrations/google-gemini
https://docs.trychroma.com/guides/embeddings

In [1]:
import os, tempfile
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
import uuid


In [2]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [3]:
config = dotenv_values("./keys/.env")
with open("./keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [4]:
client = chromadb.PersistentClient(path="./chroma_db")

In [5]:
embeddings_retriever = GoogleGenerativeAIEmbeddings(
                        model="models/embedding-001",
                        credentials=vertex_credentials,
                        google_api_key=google_api_key,
                    )

google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=google_api_key)

In [6]:
# get vectorstore from Presistem directory

In [7]:
vectorstore = Chroma(persist_directory="./chroma_db/", embedding_function=embeddings_retriever, collection_name= "forensic")

In [8]:
collection = client.get_or_create_collection(name="forensic", embedding_function=google_ef)

In [9]:
collection.count()

3807

In [35]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-pro-001", credentials=vertex_credentials, temperature=0
                )

In [36]:
metadata_field_info = [
    AttributeInfo(
        name="filename",
        description="Name of the file to search",
        type="string",
    )
]

In [37]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    "Extract information from document ",
    metadata_field_info,
)

In [41]:
# This example only specifies a filter
docs = retriever.invoke("Dame Déficit sensorial o dolor de BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf")

In [42]:
len(docs)

4

In [43]:
docs[0]

Document(page_content='SISTEMA MUSCULOESQUELÉTICO\n51 6. EVALUACIÓN DE LAS NEUROPATÍAS PERIFÉRICAS\nEn este apartado se evalúan las deficiencias de la extremidad superior relacionadas con los\ntrastornos de los nervios raquídeos (C5 a D1), el plexo braquial y los nervios periféricosprincipales. \nPara evaluar una deficiencia debida a los efectos de lesiones de los nervios periféricos es\nnecesario determinar la gravedad de la pérdida de función debida a déficit sensorial o dolory la debida a déficit motor. \nLos porcentajes de deficiencia estimados ya tienen en cuenta las manifestaciones debidas a\nlesiones de los nervios periféricos, como la limitación del movimiento, atrofia y alteracionesvasomotoras tróficas y de los reflejos. Por lo tanto, si una deficiencia deriva rigurosamente deuna lesión de un nervio periférico, el evaluador no deberá aplicar los porcentajes de defi-ciencia de los apartados anteriores de esta sección junto con los porcentajes de deficienciade este apartado, pue

In [44]:
query_prompt = ChatPromptTemplate.from_template("extrae {input} de  {filename}") # Tablas_indemnizatorias_Baremo_2024.pdf

template = """
                You are a helpful AI assistant. Answer based on the context provided. 
                context: {context}
                input: {input}
                answer:
                """
prompt = PromptTemplate.from_template(template)

In [49]:
# LCEL

In [46]:
first_chain = query_prompt | retriever 

In [47]:
second_chain = prompt | llm

In [48]:
docs = first_chain.invoke({"filename": "BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf","input" :"Déficit sensorial o dolor"})

In [50]:
docs[0]

Document(page_content='SISTEMA MUSCULOESQUELÉTICO\n51 6. EVALUACIÓN DE LAS NEUROPATÍAS PERIFÉRICAS\nEn este apartado se evalúan las deficiencias de la extremidad superior relacionadas con los\ntrastornos de los nervios raquídeos (C5 a D1), el plexo braquial y los nervios periféricosprincipales. \nPara evaluar una deficiencia debida a los efectos de lesiones de los nervios periféricos es\nnecesario determinar la gravedad de la pérdida de función debida a déficit sensorial o dolory la debida a déficit motor. \nLos porcentajes de deficiencia estimados ya tienen en cuenta las manifestaciones debidas a\nlesiones de los nervios periféricos, como la limitación del movimiento, atrofia y alteracionesvasomotoras tróficas y de los reflejos. Por lo tanto, si una deficiencia deriva rigurosamente deuna lesión de un nervio periférico, el evaluador no deberá aplicar los porcentajes de defi-ciencia de los apartados anteriores de esta sección junto con los porcentajes de deficienciade este apartado, pue

In [51]:
output = second_chain.invoke({"context": docs , "input" :"Déficit sensorial o dolor"})

In [53]:
print(output.content)

Las lesiones de los nervios periféricos que producen déficit sensorial pueden asociarse a una amplia gama de sensaciones anormales, como: anestesia, disestesia, parestesia, hiperestesia, intolerancia al frío y dolor urente intenso.

Solo el dolor o las molestias persistentes que causan una pérdida de función permanente, a pesar de un esfuerzo máximo en la rehabilitación médica y de haber transcurrido un periodo óptimo de tiempo para la adaptación psicológica, deben considerarse como una deficiencia establecida. El dolor que no cumple uno o más de los criterios anteriores no se considera valorable.

La gravedad de la pérdida de función debida a déficit sensorial se gradúa con la Tabla 21 y se relaciona con la estructura anatómica afectada y los porcentajes máximos de deficiencia por déficit sensorial de los nervios raquídeos (Tabla 23), el plexo braquial (Tabla 24) y los nervios periféricos principales (Tabla 25). 



In [54]:
complete_chain = (
    {   "filename" :itemgetter("filename"),
         "input":  itemgetter("input"),  
     "context" : first_chain
         
    } | RunnablePassthrough.assign(output=second_chain)
)

In [55]:
out = complete_chain.invoke({"filename": "BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf",  "input" :"Déficit sensorial o dolor"})

In [56]:
out.keys()

dict_keys(['filename', 'input', 'context', 'output'])

In [57]:
print(out['output'].content)

Las lesiones de los nervios periféricos que producen déficit sensorial pueden asociarse a una amplia gama de sensaciones anormales, como: anestesia, disestesia, parestesia, hiperestesia, intolerancia al frío y dolor urente intenso.

Sólo el dolor o las molestias persistentes que causan una pérdida de función permanente, a pesar de un esfuerzo máximo en la rehabilitación médica y de haber transcurrido un periodo óptimo de tiempo para la adaptación psicológica, deben considerarse como una deficiencia establecida. El dolor que no cumple uno o más de los criterios anteriores no se considera valorable.

La gravedad de la pérdida de función debida a déficit sensorial se gradúa con la Tabla 21  y se relaciona con la estructura anatómica afectada y los porcentajes máximos de deficiencia por déficit sensorial de los nervios raquídeos (Tabla 23), el plexo braquial (Tabla 24) y los nervios periféricos principales (Tabla 25). 



In [58]:
out['context']

[Document(page_content='SISTEMA MUSCULOESQUELÉTICO\n51 6. EVALUACIÓN DE LAS NEUROPATÍAS PERIFÉRICAS\nEn este apartado se evalúan las deficiencias de la extremidad superior relacionadas con los\ntrastornos de los nervios raquídeos (C5 a D1), el plexo braquial y los nervios periféricosprincipales. \nPara evaluar una deficiencia debida a los efectos de lesiones de los nervios periféricos es\nnecesario determinar la gravedad de la pérdida de función debida a déficit sensorial o dolory la debida a déficit motor. \nLos porcentajes de deficiencia estimados ya tienen en cuenta las manifestaciones debidas a\nlesiones de los nervios periféricos, como la limitación del movimiento, atrofia y alteracionesvasomotoras tróficas y de los reflejos. Por lo tanto, si una deficiencia deriva rigurosamente deuna lesión de un nervio periférico, el evaluador no deberá aplicar los porcentajes de defi-ciencia de los apartados anteriores de esta sección junto con los porcentajes de deficienciade este apartado, pu